In [1]:
library(ArchR)
library(Seurat)
library(Signac)
library(magrittr)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(ggplot2)
library(dplyr)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
TSS_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.bed"
path.atac_frag = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/HCT116/atac_fragments.tsv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/HCT116/rna_count_matrix.csv.gz"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/HCT116/HCT116/Kendall/Pairs.tsv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/HCT116/1.Genome_wide_prediction/ArchR/ArchR/"
celltype = "HCT116"

In [3]:
n.cores = 16
maxDist = 1000000

In [4]:
addArchRThreads(threads = n.cores) 
addArchRGenome("hg38")

Setting default number of Parallel threads to 16.

Setting default genome to Hg38.



Import candidate E-G pairs

In [5]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import RNA matrix

In [6]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
# matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

Map gene names

In [7]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [8]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)

make TSS GRanges

In [9]:
df.tss <- fread(TSS_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type"))
df.tss = as.data.frame(df.tss)
rownames(df.tss) = df.tss$name

In [10]:
df.tss[df.tss$strand == "+","end"] = df.tss[df.tss$strand == "+","start"]
df.tss[df.tss$strand == "-","start"] = df.tss[df.tss$strand == "-","end"]

In [11]:
df.tss = df.tss[unname(rownames(matrix.rna.rename)),]

In [12]:
gr.tss <- makeGRangesFromDataFrame(
  df.tss,
  seqnames.field = "chr",
  start.field = "start",
  end.field = "end",
  strand.field = "strand",    
  keep.extra.columns = TRUE   
)

In [13]:
gr.tss

GRanges object with 18507 ranges and 4 metadata columns:
            seqnames    ranges strand |        name     score      Ensembl_ID
               <Rle> <IRanges>  <Rle> | <character> <integer>     <character>
     FAM87B     chr1    817370      + |      FAM87B         0 ENSG00000177757
  LINC00115     chr1    827522      - |   LINC00115         0 ENSG00000225880
  LINC01128     chr1    827590      + |   LINC01128         0 ENSG00000228794
     FAM41C     chr1    876802      - |      FAM41C         0 ENSG00000230368
     SAMD11     chr1    925740      + |      SAMD11         0 ENSG00000187634
        ...      ...       ...    ... .         ...       ...             ...
     NLGN4Y     chrY  14523745      + |      NLGN4Y         0 ENSG00000165246
     TTTY14     chrY  19077547      - |      TTTY14         0 ENSG00000176728
      KDM5D     chrY  19744939      - |       KDM5D         0 ENSG00000012817
     TTTY10     chrY  20519228      - |      TTTY10         0 ENSG00000229236
     EI

Create an ArchRProject

In [14]:
ArrowFiles <- createArrowFiles(
  inputFiles = c(path.atac_frag),
  sampleNames = c(celltype),
  validBarcodes = list(HCT116 = colnames(matrix.rna.rename)),
  addTileMat = T,
  addGeneScoreMat = F,
  force = T
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-273ebf78442f20-Date-2025-03-22_Time-16-25-08.50193.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2025-03-22 16:25:09.042546 : Batch Execution w/ safelapply!, 0 mins elapsed.

(HCT116 : 1 of 1) Checking if completed file exists!

2025-03-22 16:25:09.107715 : (HCT116 : 1 of 1) Arrow Exists! Overriding since force = TRUE!, 0.001 mins elapsed.

(HCT116 : 1 of 1) Determining Arrow Method to use!

2025-03-22 16:25:09.262686 : (HCT116 : 1 of 1) Reading In Fragments from inputFiles (readMethod = tabix), 0.004 mins elapsed.

2025-03-22 16:25:09.272751 : (HCT116 : 1 of 1) Tabix Bed To Temporary File, 0.004 mins elapsed.

2025-03-22 16:26:24.357439 : (HCT116 : 1 of 1) Successful creation of Temporary File, 1.255 mins elapsed.

2025-03-22 16:26:24.367574 : (HCT116 : 1 of 1) Creating ArrowFile From Temporar

In [15]:
ArrowFiles

[1] "HCT116.arrow"

In [16]:
dir.create(paste(dir.output,"ArchR",sep="/"),recursive = T)

Warning message in dir.create(paste(dir.output, "ArchR", sep = "/"), recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/HCT116/1.Genome_wide_prediction/ArchR/ArchR//ArchR' already exists”


In [17]:
obj.ArchR <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = paste(dir.output,"ArchR",sep="/"),
  copyArrows = TRUE 
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...

1 


Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 


Getting Cell Metadata...

1 


Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--'

In [18]:
obj.ArchR


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/HCT116/1.Genome_wide_prediction/ArchR/ArchR/ArchR 
samples(1): HCT116
sampleColData names(1): ArrowFiles
cellColData names(13): Sample TSSEnrichment ... nDiFrags BlacklistRatio
numberOfCells(1): 4820
medianTSS(1): 13.9535
medianFrags(1): 7478.5

In [19]:
obj.ArchR = addIterativeLSI(
    ArchRProj = obj.ArchR,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI"
)

Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-273ebf9b4b9e-Date-2025-03-22_Time-16-33-35.300146.log
If there is an issue, please report to github with logFile!

2025-03-22 16:33:35.853881 : Computing Total Across All Features, 0 mins elapsed.

2025-03-22 16:33:36.945175 : Computing Top Features, 0.019 mins elapsed.

###########
2025-03-22 16:33:40.300735 : Running LSI (1 of 2) on Top Features, 0.075 mins elapsed.
###########

2025-03-22 16:33:40.31784 : Creating Partial Matrix, 0.075 mins elapsed.

2025-03-22 16:33:49.30453 : Computing LSI, 0.225 mins elapsed.

2025-03-22 16:34:30.701454 : Identifying Clusters, 0.915 mins elapsed.

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
2025-03-22 16:34:36.869205 : Identified 6 Clusters, 1.017 mins elapsed.

2025-03-22 16:34:36.88252 : Saving LSI Iteration, 1.018 mins elapsed.

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by ‘spam’

Found mor


************************************************************
2025-03-22 16:34:53.311017 : ERROR Found in .saveIteration for  
LogFile = ArchRLogs/ArchR-addIterativeLSI-273ebf9b4b9e-Date-2025-03-22_Time-16-33-35.300146.log

<simpleError in g$grobs[[legend]]: no such index at level 2
>

************************************************************



2025-03-22 16:34:53.313029 : Creating Cluster Matrix on the total Group Features, 1.291 mins elapsed.

2025-03-22 16:35:10.03345 : Computing Variable Features, 1.57 mins elapsed.

###########
2025-03-22 16:35:10.210411 : Running LSI (2 of 2) on Variable Features, 1.573 mins elapsed.
###########

2025-03-22 16:35:10.227274 : Creating Partial Matrix, 1.573 mins elapsed.

2025-03-22 16:35:20.364643 : Computing LSI, 1.742 mins elapsed.

2025-03-22 16:36:04.260698 : Finished Running IterativeLSI, 2.474 mins elapsed.



In [20]:
obj.ArchR = addPeakSet(obj.ArchR,
                       peakSet = reduce(pairs.E2G))

In [21]:
obj.ArchR = addPeakMatrix(obj.ArchR)

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-273ebf27b41c52-Date-2025-03-22_Time-16-36-07.793911.log
If there is an issue, please report to github with logFile!

2025-03-22 16:36:08.108703 : Batch Execution w/ safelapply!, 0 mins elapsed.

2025-03-22 16:36:09.043859 : Adding HCT116 to PeakMatrix for Chr (1 of 23)!, 0.015 mins elapsed.

2025-03-22 16:36:17.52651 : Adding HCT116 to PeakMatrix for Chr (2 of 23)!, 0.156 mins elapsed.

2025-03-22 16:36:24.748023 : Adding HCT116 to PeakMatrix for Chr (3 of 23)!, 0.277 mins elapsed.

2025-03-22 16:36:31.990789 : Adding HCT116 to PeakMatrix for Chr (4 of 23)!, 0.397 mins elapsed.

2025-03-22 16:36:38.219588 : Adding HCT116 to PeakMatrix for Chr (5 of 23)!, 0.501 mins elapsed.

2025-03-22 16:36:44.735247 : Adding HCT116 to PeakMatrix for Chr (6 of 23)!, 0.61 mins elapsed.

2025-03-22 16:36:51.058805 : Adding HCT116 to PeakMatrix for Chr (7 of 23)!, 0.715 mins elapsed.

2025-03-22 16:36:57.669287 : Adding HCT116 to PeakMatrix for Chr (8 of 2

In [22]:
matrix.rna.rename2 = matrix.rna.rename
colnames(matrix.rna.rename2) = paste(celltype,colnames(matrix.rna.rename),sep="#")

In [23]:
seRNA <- SummarizedExperiment(
  assays = SimpleList(counts = matrix.rna.rename2),
  rowRanges = gr.tss
)

In [24]:
seRNA = sort.GenomicRanges(sortSeqlevels(seRNA), 
                           ignore.strand = TRUE)

In [25]:
seRNA

class: RangedSummarizedExperiment 
dim: 18507 4825 
metadata(0):
assays(1): counts
rownames(18507): FAM87B LINC00115 ... TTTY10 EIF1AY
rowData names(4): name score Ensembl_ID gene_type
colnames(4825): HCT116#e18l1_AAACCGGTCGCCTGTG-1
  HCT116#e18l1_AAACCGGTCGTTGTCA-1 ... HCT116#e18l4_TTTCAGTAGTGAGGCC-1
  HCT116#e18l4_TTTGCTCTCCGCCATG-1
colData names(0):

In [26]:
obj.ArchR = addGeneExpressionMatrix(obj.ArchR,
                                    seRNA = seRNA,
                                    strictMatch = T)

ArchR logging to : ArchRLogs/ArchR-addGeneExpressionMatrix-273ebf43bb5e3c-Date-2025-03-22_Time-16-38-36.197716.log
If there is an issue, please report to github with logFile!

Overlap w/ scATAC = 1

2025-03-22 16:38:36.65379 : 

Overlap Per Sample w/ scATAC : HCT116=4820

2025-03-22 16:38:36.665195 : 

2025-03-22 16:38:37.974174 : Batch Execution w/ safelapply!, 0 mins elapsed.

2025-03-22 16:38:38.646524 : Adding HCT116 to GeneExpressionMatrix for Chr (1 of 23)!, 0.011 mins elapsed.

2025-03-22 16:38:44.244678 : Adding HCT116 to GeneExpressionMatrix for Chr (2 of 23)!, 0.105 mins elapsed.

2025-03-22 16:38:49.506628 : Adding HCT116 to GeneExpressionMatrix for Chr (3 of 23)!, 0.192 mins elapsed.

2025-03-22 16:38:56.908768 : Adding HCT116 to GeneExpressionMatrix for Chr (4 of 23)!, 0.316 mins elapsed.

2025-03-22 16:39:02.065362 : Adding HCT116 to GeneExpressionMatrix for Chr (5 of 23)!, 0.402 mins elapsed.

2025-03-22 16:39:07.575245 : Adding HCT116 to GeneExpressionMatrix for Chr (6 

Run ArchR prediction

In [27]:
start_time <- Sys.time()
obj.ArchR = addPeak2GeneLinks(
    ArchRProj = obj.ArchR,
    reducedDims = "IterativeLSI",
    useMatrix = "GeneExpressionMatrix",
    maxDist = maxDist,
    threads = n.cores
)
end_time <- Sys.time()
execution_time <- end_time - start_time

ArchR logging to : ArchRLogs/ArchR-addPeak2GeneLinks-273ebfdc093e-Date-2025-03-22_Time-16-40-59.44763.log
If there is an issue, please report to github with logFile!

2025-03-22 16:40:59.852361 : Getting Available Matrices, 0.007 mins elapsed.

No predictionScore found. Continuing without predictionScore!

2025-03-22 16:41:00.221672 : Filtered Low Prediction Score Cells (0 of 4820, 0), 0 mins elapsed.

2025-03-22 16:41:00.314559 : Computing KNN, 0.002 mins elapsed.

2025-03-22 16:41:00.442999 : Identifying Non-Overlapping KNN pairs, 0.004 mins elapsed.

2025-03-22 16:41:02.225569 : Identified 490 Groupings!, 0.034 mins elapsed.

2025-03-22 16:41:02.263244 : Getting Group RNA Matrix, 0.034 mins elapsed.

2025-03-22 16:41:19.778969 : Getting Group ATAC Matrix, 0.326 mins elapsed.

2025-03-22 16:41:56.07273 : Normalizing Group Matrices, 0.931 mins elapsed.

2025-03-22 16:42:01.985246 : Finding Peak Gene Pairings, 1.03 mins elapsed.

2025-03-22 16:42:02.785524 : Computing Correlations, 1.0

In [28]:
execution_time

Time difference of 1.302415 mins

In [29]:
p2g = metadata(obj.ArchR@peakSet)$Peak2GeneLinks

In [30]:
metadata(p2g)[[1]]$PeakName = paste(seqnames(metadata(p2g)[[1]]),
                                    ranges(metadata(p2g)[[1]]),sep="-")

In [31]:
p2g$PairName = paste(metadata(p2g)[[1]][p2g$idxATAC]$PeakName,
                     metadata(p2g)[[2]][p2g$idxRNA]$name,
                     sep = "_")

In [32]:
rownames(p2g) = p2g$PairName
p2g

DataFrame with 3432522 rows and 7 columns
                                idxATAC    idxRNA Correlation       FDR
                              <integer> <integer>   <numeric> <numeric>
chr1-10081-10420_FAM87B               1         1  0.14751150 0.0174716
chr1-91097-91445_FAM87B               2         1 -0.07298123 0.3247146
chr1-180701-181904_FAM87B             3         1  0.00180292 0.9874658
chr1-267895-268118_FAM87B             4         1 -0.01362302 0.8941518
chr1-586092-586306_FAM87B             5         1 -0.03564690 0.6851630
...                                 ...       ...         ...       ...
chrX-156003477-156004102_IL9R    153350     18493 -0.03705767  0.670750
chrX-156005417-156005618_IL9R    153351     18493 -0.05091176  0.530522
chrX-156019909-156020179_IL9R    153352     18493 -0.03818377  0.659411
chrX-156029903-156030297_IL9R    153353     18493  0.04229849  0.617300
chrX-156030522-156030719_IL9R    153354     18493 -0.00732478  0.945949
                      

In [33]:
pairs.E2G.res = pairs.E2G[pairs.E2G$PairName %in% p2g$PairName]
mcols(pairs.E2G.res)[,c("Correlation","FDR","VarQATAC","VarQRNA")] = p2g[pairs.E2G.res$PairName,c("Correlation","FDR","VarQATAC","VarQRNA")]

In [34]:
pairs.E2G.res

GRanges object with 2843272 ranges and 7 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100034595-100034763      * |         AGL
        [2]     chr1 100034595-100034763      * |      CDC14A
        [3]     chr1 100034595-100034763      * |         DBT
        [4]     chr1 100034595-100034763      * |        DPH5
        [5]     chr1 100034595-100034763      * |       EXTL2
        ...      ...                 ...    ... .         ...
  [2843268]     chrX     9911462-9911841      * |        WWC3
  [2843269]     chrX     9995795-9996095      * |       CLCN4
  [2843270]     chrX     9995795-9996095      * |       FAM9B
  [2843271]     chrX     9995795-9996095      * |       TBL1X
  [2843272]     chrX     9995795-9996095      * |        WWC3
                          PeakName               PairName Correlation       FDR
                       <character>            <characte

Save results

In [35]:
saveRDS(obj.ArchR,
        paste(dir.output,"obj.ArchR.rds",sep = "/"))
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "Correlation",
                         "FDR",
                         "VarQATAC",
                         "VarQRNA")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
pairs.E2G.res
df.output

GRanges object with 2843272 ranges and 7 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100034595-100034763      * |         AGL
        [2]     chr1 100034595-100034763      * |      CDC14A
        [3]     chr1 100034595-100034763      * |         DBT
        [4]     chr1 100034595-100034763      * |        DPH5
        [5]     chr1 100034595-100034763      * |       EXTL2
        ...      ...                 ...    ... .         ...
  [2843268]     chrX     9911462-9911841      * |        WWC3
  [2843269]     chrX     9995795-9996095      * |       CLCN4
  [2843270]     chrX     9995795-9996095      * |       FAM9B
  [2843271]     chrX     9995795-9996095      * |       TBL1X
  [2843272]     chrX     9995795-9996095      * |        WWC3
                          PeakName               PairName Correlation       FDR
                       <character>            <characte

chr,start,end,TargetGene,CellType,Correlation,FDR,VarQATAC,VarQRNA
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100034595,100034763,AGL,HCT116,-0.0886901403,0.208551811,0.3392934,0.7771049
chr1,100034595,100034763,CDC14A,HCT116,0.0704271925,0.346397921,0.3392934,0.7359541
chr1,100034595,100034763,DBT,HCT116,0.0310577472,0.730995946,0.3392934,0.5666468
chr1,100034595,100034763,DPH5,HCT116,0.0783784441,0.281610661,0.3392934,0.7432542
chr1,100034595,100034763,EXTL2,HCT116,-0.0433380562,0.606807519,0.3392934,0.7516898
chr1,100034595,100034763,FRRS1,HCT116,-0.0440303083,0.599852960,0.3392934,0.6593306
chr1,100034595,100034763,MFSD14A,HCT116,-0.0267752615,0.773023884,0.3392934,0.7187584
chr1,100034595,100034763,PLPPR4,HCT116,-0.0446559226,0.593473464,0.3392934,0.2750230
chr1,100034595,100034763,RTCA,HCT116,0.1656950857,0.006051716,0.3392934,0.6582491


In [36]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

Random number generation:
 RNG:     L'Ecuyer-CMRG 
 Normal:  Inversion 
 Sample:  Rejection 
 
locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] uwot_0.1.16           